# Background Set Gene Selection

In [1]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Inputs

In [2]:
df = pd.read_hdf('/mnt/data/Objects/tcga_gtex_tpm_norm_filt.hd5')
df.head(2)

,id,tissue,subtype,tumor,label,RP11-40C6.2,IGHG1,IGKC,KRT5,IGLC2,...,IGHEP1,RP11-44N21.4,GAPDHP61,P2RX5-TAX1BP3,RP11-408H20.3,RP13-726E6.2,CTB-43E15.1,ZNF878,OR2H2,RP4-530I15.9
GTEX-UTHO-1226-SM-3GAEE,GTEX-UTHO-1226-SM-3GAEE,Heart,Heart_Atrial_Appendage,no,gtex,0.000000,10.302941,10.712460,2.750618,7.780669,...,0.000000,0.0,0.0,0.150556,0.0,0.000000,0.0,0.070387,0.790754,0.815589
GTEX-146FH-1726-SM-5QGQ2,GTEX-146FH-1726-SM-5QGQ2,Esophagus,Esophagus_Mucosa,no,gtex,2.970847,8.778885,10.445933,12.393268,8.440056,...,0.176318,0.0,0.0,0.411426,0.0,0.176318,0.0,0.622930,0.042644,0.378503


Subsets

In [3]:
gtex = df[df.label == 'gtex']
normal = df[df.label == 'tcga-normal']
tumor = df[df.label == 'tcga-tumor']

# LDA GTEx

In [7]:
genes = df.columns[5:]
clf = LinearDiscriminantAnalysis()
clf.fit(gtex[genes], gtex.tissue)

/home/ubuntu/anaconda3/envs/pan-normal/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/home/ubuntu/anaconda3/envs/pan-normal/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [9]:
clf.coef_.shape

(31, 26544)

In [19]:
gtex_coef = pd.DataFrame({'gene': genes, 'coef': np.median(clf.coef_, axis=0)})
gtex_coef = gtex_coef.sort_values('coef', ascending=False)
gtex_coef.geneee[:50]

,gene,coef
19597,LAMTOR5,328292.839159
17542,FAU,327657.246919
25447,HMGN2P17,315865.812769
11381,RPS27,309922.845085
12872,EIF3B,298222.547032


# SelectKBest

In [20]:
from sklearn.feature_selection import SelectKBest

In [23]:
k = SelectKBest(k=50)
kbest = k.fit_transform(gtex[genes], gtex.tissue)
feature_names = [df.columns[5:][i] for i in k.get_support(indices=True)]